![STScI Logo](../../../_static/stsci_header.png)

<a id="titleImageReduction"></a>
# ACS Image Reduction for Subarray Data

## Table of Contents

[Introduction](#intro_ID) <br>
[Imports](#imports) <br>
[Download the Data](#download) <br>
[File Information](#file) <br>
[1. De-striping and CTE Correction of Post-SM4 Subarray Observations](#_postsm4) <br>
[2. Reducing Custom Subarray Data](#_custom) <br>
[About this Notebook](#about)


<a id="intro_ID"></a>
## Introduction

Subarray data requires different considerations than working with full frame images. This notebook will guide you through working with standard post-SM4 subarray data and custom subarray data. 

After Servicing Mission 4 (SM4; May 2009), the installation of the ASIC during the repair of ACS introduced $1/f$ noise in all ACS images. In the calacs pipeline, only full-frame data have this striping removed. To correct subarray data, the alternative acs_destripe_plus pipeline must be used, which will apply all of calibration steps normally performed by calacs in addition to de-striping for subarray data. De-striping is only possible for 2K subarrays after SM4 until Cycle 24, after which a change to the flight software makes all subarrays eligible for de-striping.<!-- </div> -->

### This tutorial will show you how to handle...

#### 1. [Post-SM4 Subarray Data](#_postsm4) 

* Update header keywords.
* Clean Subarray images with the option to correct CTE losses. 
* Update WCS solution.

#### 2. [Custom Subarray Data](#_custom)

* Use `AstroDrizzle` with ASN files to combine images.

<a id="imports"></a>
## Imports

Here we list the Python packages used in this notebook. Links to the documentation for each module is provided for convenience.

| Package Name     | module           | docs          | used for    |
|------------------|:-----------------|:-------------:|:------------|
| `os`             | `system`           | <a class="divlink" href="https://docs.python.org/2/library/os.html#os.system">link</a>|command line input|
| `os`             | `environ`          | <a class="divlink" href="https://docs.python.org/2/library/os.html#os.environ">link</a>| setting environments |
|`shutil`          | `rmtree`           | <a class="divlink" href="https://docs.python.org/2/library/shutil.html#shutil.rmtree">link</a>| remove directory tree |
|`glob`            | `glob`             | <a class="divlink" href="https://docs.python.org/2/library/glob.html#glob.glob">link</a>| search for files based on Unix shell rules |
|`astroquery.mast` |`Observations`      | <a class="divlink" href="https://astroquery.readthedocs.io/en/latest/mast/mast.html">link</a>| download data from MAST |
|`astropy.io`      | `fits`             | <a class="divlink" href="http://docs.astropy.org/en/stable/io/fits/">link</a>| access and update fits files |
|`astropy.table`   | `Table`            | <a class="divlink" href="http://docs.astropy.org/en/stable/table/">link</a>| constructing and editing in a tabular format |
|`stwcs`           |`updatewcs`         | <a class="divlink" href="https://stwcs.readthedocs.io/en/latest/updatewcs_ui.html">link</a>| update wcs solution |
|`acstools`       |`acs_destripe_plus`| <a class="divlink" href="https://acstools.readthedocs.io/en/latest/api/acstools.acs_destripe_plus.destripe_plus.html">link</a>| destripe acs images and optionally CTE correct |
|`acstools`       |`utils_calib`| <a class="divlink" href="https://acstools.readthedocs.io/en/stable/utils_calib.html">link</a>| check for settings in oscntab |

In [1]:
import os
import shutil

from astroquery.mast import Observations
from astropy.io import fits
from astropy.table import Table

from stwcs import updatewcs
from acstools import (acs_destripe_plus, utils_calib)
from IPython.display import clear_output

/home/runner/micromamba/envs/ci-env/lib/python3.11/site-packages/acstools/utils_findsat_mrt.py:24: UserWarning: skimage not installed. MRT calculation will not work: ModuleNotFoundError("No module named 'skimage'")
  warnings.warn(f'skimage not installed. MRT calculation will not work: {repr(e)}')  # noqa
/home/runner/micromamba/envs/ci-env/lib/python3.11/site-packages/acstools/utils_findsat_mrt.py:30: UserWarning: scipy not installed. Kernel generation will not work
  warnings.warn('scipy not installed. Kernel generation will not work')
/home/runner/micromamba/envs/ci-env/lib/python3.11/site-packages/acstools/utils_findsat_mrt.py:37: UserWarning: matplotlib not found, plotting is disabled
  warnings.warn('matplotlib not found, plotting is disabled')
/home/runner/micromamba/envs/ci-env/lib/python3.11/site-packages/acstools/findsat_mrt.py:117: UserWarning: matplotlib not found, plotting is disabled
  warnings.warn('matplotlib not found, plotting is disabled')


Here we set environment variables for later use with the Calibration Reference Data System (CRDS). 

In [2]:
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = './crds_cache'
os.environ['jref'] = './crds_cache/references/hst/acs/'

<a id="download"></a>
## Download the Data

Here we download all of the data required for this notebook. This is an important step! Some of the image processing steps require all relevant files to be in the working directory. We recommend working with a brand new directory for every new set of data.

### [GO Proposal 14511](https://stdatu.stsci.edu/proposal_search.php?mission=hst&id=14511): "ACS CCD Stability Monitor"

Our data for the first example comes from a routine calibration program of two images using the post-Cycle 24 WFC1A-1K 1024 x 2072 subarray. We will only use the data associated with the observation id `JD5702JWQ`.

Using the python package `astroquery`, we can download files from the [MAST](http://archive.stsci.edu) archive.

<div class="alert alert-danger">
<b>MAY CHANGE:</b> The argument "mrp_only" stands for "minimum recommended products only". It currently needs to be set to False, although in the future, False is intended to be set as the default and can be left out.
</div>

In [3]:
obs_table = Observations.query_criteria(proposal_id=14511, obs_id='JD5702JWQ')

dl_table = Observations.download_products(obs_table['obsid'],
                                          productSubGroupDescription=['RAW'],
                                          mrp_only=False)

 [Done]


We'll use the package `os` to put all of these files in our working directory for convenience.

In [4]:
for row in dl_table:
    oldfname = row['Local Path']
    newfname = os.path.basename(oldfname)
    os.rename(oldfname, newfname)

### [GO Proposal 10206](https://stdatu.stsci.edu/proposal_search.php?mission=hst&id=10206): "What drives the outflows in powerful radio galaxies?"

For the second example, we will use a ramp filter observation of the galaxy PLS1345+12 (*HST* proposal 10206). The association name is J92SA0010, and we will only use one image in the association: J92SA0W6Q.

Again, we use `astroquery`, to download files from the [MAST](http://archive.stsci.edu) archive. Along with the raw images, we will also need the spt (telemetry) files to reduce our custom subarray data.

In [5]:
obs_table = Observations.query_criteria(proposal_id=10206, obs_id='J92SA0010')

dl_table = Observations.download_products(obs_table['obsid'],
                                          productSubGroupDescription=['RAW', 'SPT'],
                                          mrp_only=False)

 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


 [Done]


Again, we'll use the package `os` to put all of these files in our working directory for convenience.

In [6]:
for row in dl_table:
    oldfname = row['Local Path']
    newfname = os.path.basename(oldfname)
    os.rename(oldfname, newfname)

Now that all of our files are in the current working directory, we delete the leftover MAST file structure.

In [7]:
shutil.rmtree('mastDownload')

<a id="file"></a>
## File Information

The structure of the fits files from ACS may be different depending on what kind of observation was made. For more information, refer to Section 2.2 of the [ACS Data Handbook](https://hst-docs.stsci.edu/display/ACSIHB).

#### Raw Files

A standard raw image file from a subarray has the same structure as you'd expect from full frame observation from ACS/WCS.

| Ext    | Name             | Type         | Contains                                               |
|--------|------------------|--------------|:-------------------------------------------------------|
|<b>0</b>| PRIMARY          | (PrimaryHDU) | Meta-data related to the entire file.                  |
|<b>1</b>| SCI (Image)      | (ImageHDU)   | Raw image data.                                        |
|<b>2</b>| ERR (Error)      | (ImageHDU)   | Error array.                                           |
|<b>3</b>| DQ (Data Quality)| (ImageHDU)   | Data quality array.                                    |

#### SPT Files

SPT files contain telemetry and engineering data from the telescope.

| Ext    | Name             | Type         | Contains                                               |
|--------|------------------|--------------|:-------------------------------------------------------|
|<b>0</b>| PRIMARY          | (PrimaryHDU) | Meta-data related to the entire file.                  |
|<b>1</b>| UDL (Image)      | (ImageHDU)   | Raw image data.                                        |

You can always use `.info()` on an HDUlist for an overview of the structure

In [8]:
with fits.open('j92sa0w6q_raw.fits') as hdulist:
    hdulist.info()
    
with fits.open('j92sa0w6q_spt.fits') as hdulist:
    hdulist.info()

Filename: j92sa0w6q_raw.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     216   ()      
  1  SCI           1 ImageHDU        97   (1180, 1200)   int16 (rescales to uint16)   
  2  ERR           1 ImageHDU        49   ()      
  3  DQ            1 ImageHDU        43   ()      


Filename: j92sa0w6q_spt.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     341   ()      
  1  UDL           1 ImageHDU       305   (965,)   int16   


<a id="_postsm4"></a>
## 1. De-striping and CTE Correction of Post-SM4 Subarray Observations

### Download Calibration Files: Update Header Keywords

We can call the [Calibration Reference Data System](https://hst-crds.stsci.edu/) (CRDS) to get the associated calibration files for this image.

First we will need to turn on the CTE correction switch in the primary header. Turning on this switch will notify the [CRDS](https://hst-crds.stsci.edu) `bestrefs` tool to add the names of the CTE correction parameters table '`PCTETAB`' and CTE-corrected dark current image '`DRKCFILE`' reference files to the header.

In [9]:
sbc_fits = 'jd5702jwq_raw.fits'

with fits.open(sbc_fits, mode='update') as hdulist:
    hdulist[0].header['PCTECORR'] = 'PERFORM'

#### Download Calibration Files

The CRDS program can be run from the terminal to download the files specific to your observation. This program is packaged with astroconda. 

The command line input to download our files for `jd5702jwq_raw.fits` is as follows:

    crds bestrefs --files jd5702jwq_raw.fits --sync-references=1 --update-bestrefs
    
Here, we use the python package `os` to run this command.

In [10]:
!crds bestrefs --files {sbc_fits} --sync-references=1 --update-bestrefs
clear_output()

#### Clean Subarray Images

Next we will run the [acs_destripe_plus](https://acstools.readthedocs.io/en/latest/api/acstools.acs_destripe_plus.destripe_plus.html#acstools.acs_destripe_plus.destripe_plus) code on our image. This will execute all of the calibration steps that are set to 'PERFORM' in the primary header of the FITS file. 

The `acs_destripe_plus` code will produce the FLT file is the calibrated output product from `CALACS`, `jd5702jmq_flt.fits`. With the CTE correction turned on, an FLC file will also be produced, which is the same as the FLT file but with the CTE correction applied.

In [11]:
acs_destripe_plus.destripe_plus(sbc_fits, cte_correct=True)

git tag: 0090c701-dirty
git branch: HEAD
HEAD @: 0090c701d894003cfc690e9f8d5fde81e6939090
Trying to open jd5702jwq_raw.fits...


Read in Primary header from jd5702jwq_raw.fits...


CALBEG*** ACSCCD -- Version 10.4.0 (07-May-2024) ***
Begin    12-Nov-2025 21:20:57 UTC
Input    jd5702jwq_raw.fits
Output   jd5702jwq_blv_tmp.fits
Trying to open jd5702jwq_raw.fits...
Read in Primary header from jd5702jwq_raw.fits...
APERTURE WFC1A-2K
FILTER1 F606W
FILTER2 CLEAR2L
DETECTOR WFC


CCDTAB   jref$72m1821dj_ccd.fits
CCDTAB   PEDIGREE=inflight
CCDTAB   DESCRIP =CCD table with updated readnoise values for CCDGAIN=2.-------------
CCDTAB   DESCRIP =July 2009


DQICORR  PERFORM
DQITAB   jref$25g1256nj_bpx.fits
DQITAB   PEDIGREE=INFLIGHT 30/03/2002 30/05/2002
DQITAB   DESCRIP =FITS reference table for bad pixel locations on ACS WFC
DQICORR  COMPLETE


BIASCORR PERFORM
BIASFILE jref$6641832dj_bia.fits
BIASFILE PEDIGREE=INFLIGHT 08/02/2017 08/03/2017
BIASFILE DESCRIP =Standard full-frame bias for data taken after Feb 07 2017 11:26:21-


BIASCORR COMPLETE


BLEVCORR PERFORM
OSCNTAB  jref$17717071j_osc.fits
OSCNTAB  PEDIGREE=GROUND
OSCNTAB  DESCRIP =New OSCNTAB which includes entries for all subarrays.--------------
This subarray data is supported for the bias shift correction.
BLEVCORR PERFORM
Performing bias-shift correction for subarray data.
BLEVCORR PERFORM


Bias shift correcting for bias level in Amp A of 4197.2216 electrons.
BLEVCORR COMPLETE
Supported Subarray adjusted Darktime: 43.094452 for aperture: WFC1A-2K

DARKTIME from SCI header: 43.094452  Offset from CCDTAB: 0.000000  Final DARKTIME: 43.094452

Full-well saturation flagging being performed for imset 1.

Subarray full-well saturation image flagging step being performed.

Subarray full-well saturation image flagging step done.


    Uncertainty array initialized,
    readnoise =4.35,
    gain =2.02,
   default bias levels =4242,


SINKCORR PERFORM
SNKCFILE jref$25e1542jj_snk.fits
SNKCFILE PEDIGREE=INFLIGHT 08/02/2017 08/03/2017
SNKCFILE DESCRIP =Sink pixel reference file ----------------------------
SINKCORR COMPLETE


INFO:acs_destripe:Processing jd5702jwq_blv_tmp.fits


...

INFO:acs_destripe:jd5702jwq_blv_tmp_strp.fits created


git tag: 0090c701-dirty
git branch: HEAD
HEAD @: 0090c701d894003cfc690e9f8d5fde81e6939090
Setting max threads to 4 out of 4 available
Trying to open jd5702jwq_blv_tmp.fits...
Read in Primary header from jd5702jwq_blv_tmp.fits...


CALBEG*** ACSCTE -- Version 10.4.0 (07-May-2024) ***
Begin    12-Nov-2025 21:20:59 UTC
Input    jd5702jwq_blv_tmp.fits
Output   jd5702jwq_blc_tmp.fits
Trying to open jd5702jwq_blv_tmp.fits...
Read in Primary header from jd5702jwq_blv_tmp.fits...
APERTURE WFC1A-2K
FILTER1 F606W
FILTER2 CLEAR2L
DETECTOR WFC


CCDTAB   jref$72m1821dj_ccd.fits
CCDTAB   PEDIGREE=inflight
CCDTAB   DESCRIP =CCD table with updated readnoise values for CCDGAIN=2.-------------
CCDTAB   DESCRIP =July 2009


PCTECORR PERFORM
PCTEFILE jref$8ch1518tj_cte.fits
PCTEFILE PEDIGREE=INFLIGHT 01/03/2002 22/07/2010
PCTEFILE DESCRIP =Parameters needed for gen3 pixel-based CTE correction -----------
Trying to open jref$8ch1518tj_cte.fits...
Read in Primary header from jref$8ch1518tj_cte.fits...
(pct

(pctecorr) Time taken to smooth image: 2.15(s) with 4 threads
(pctecorr) ...complete.
(pctecorr) Creating charge trap image...
(pctecorr) Time taken to populate pixel trap map image: 0.01(s) with 4 threads
(pctecorr) ...complete.
(pctecorr) Running correction algorithm...


INFO:acs_destripe_plus:Done.
FLT: jd5702jwq_flt.fits
FLC: jd5702jwq_flc.fits



(pctecorr) ...complete.
(pctecorr) Total count difference (corrected-raw) incurred from correction: 530760.750000 (0.575928%)
(pctecorr) CTE run time for current chip: 325.02(s) with 4 procs/threads

PCTECORR COMPLETE


End      12-Nov-2025 21:26:27 UTC


*** ACSCTE complete ***
git tag: 0090c701-dirty
git branch: HEAD
HEAD @: 0090c701d894003cfc690e9f8d5fde81e6939090
Trying to open jd5702jwq_blv_tmp.fits...
Read in Primary header from jd5702jwq_blv_tmp.fits...


CALBEG*** ACS2D -- Version 10.4.0 (07-May-2024) ***
Begin    12-Nov-2025 21:26:27 UTC
Input    jd5702jwq_blv_tmp.fits
Output   jd5702jwq_flt.fits
Warning    Output file `jd5702jwq_flt.fits' already exists.
ERROR:    Error processing jd5702jwq_blv_tmp.fits.
ERROR:    Error processing jd5702jwq_blv_tmp.fits.
ERROR:             status = 1021
git tag: 0090c701-dirty
git branch: HEAD
HEAD @: 0090c701d894003cfc690e9f8d5fde81e6939090
Trying to open jd5702jwq_blc_tmp.fits...
Read in Primary header from jd5702jwq_blc_tmp.fits...


CALBE

#### Correct the WCS

The subarray products produced by this process do not have the proper WCS information in the header. The WCS is normally updated by the pipeline via an additional call to AstroDrizzle. Here, we can manually update the WCS of our FLC product using `stwcs.updatewcs`.

In [12]:
updatewcs.updatewcs('jd5702jwq_flc.fits', use_db=False)

INFO:astropy:
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                


INFO:astropy:
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
Using 2015-calibrate

Using 2015-calibrated VAFACTOR-corrected TDD correction...
- IDCTAB: Distortion model from row 443 for chip 1 : F606W


['jd5702jwq_flc.fits']

<a id="_custom"></a>
## 2. Reducing Custom Subarray Data

### Download Calibration Files

Like before, we can use CRDS to get the associated calibration files for this image.

In [13]:
sbc_fits = 'j92sa0w6q_raw.fits'
!crds bestrefs --files {sbc_fits} --sync-references=1 --update-bestrefs
clear_output()

### Access OSCN Table

The name in the header is of the format `ref\$oscn_name.fits`, therefore we need to split the string on the '$' character.

In [14]:
prihdr = fits.getheader(sbc_fits)
scihdr = fits.getheader(sbc_fits, ext=1)

We also want to retrieve the `OSCNTAB` reference file from the JREF directory. We can get the name of the file from the primary header of the image.

In [15]:
oscn_name = prihdr['OSCNTAB'].split('$')[-1]
path_to_oscn = os.path.join(os.environ['jref'], oscn_name)

print(path_to_oscn)

./crds_cache/references/hst/acs/17717071j_osc.fits


The `utils_calib.check_oscntab` from [acstools](http://acstools.readthedocs.io) checks the `OSCNTAB` file if any entry matches the combination of readout amplifier, image size, and number of bias prescan columns for a given subarray observation. We will need to use several header keyword values to test if a subarray is in `OSCNTAB`.

In [16]:
oscnrec = fits.getdata(path_to_oscn)
oscnhdr = fits.getheader(path_to_oscn)

oscntable = Table(oscnrec)

The raw image has 1180 columns and 1200 rows, which does not correspond to any entry in the `OSCNTAB` file, but a visual examination of the image shows that it contains bias prescan columns.

### Get the Bias Prescan Information

From the science image PRI and SCI extension headers that we opened earlier, we can get the information about the readout amplifier and dimensions of the image.

In [17]:
amp = prihdr['CCDAMP']
xsize = scihdr['NAXIS1']
ysize = scihdr['NAXIS2']

print(xsize, ysize)

1180 1200


To get information on the number of prescan columns (if any), we need to access the SPT first extension header.

In [18]:
spthdr = fits.getheader('j92sa0w6q_spt.fits', ext=1)

leading = spthdr['OVERSCNA']
trailing = spthdr['OVERSCNB']

Finally, we check if this subarray definition is in the OSCNTAB file. The code returns a boolean result, which we have saved as the variable supported, to describe this.

In [19]:
supported = utils_calib.check_oscntab(path_to_oscn, amp, xsize, ysize, leading, trailing)
print(supported)

True


### Update OSCNTAB

Now that we have confirmed that the `OSCNTAB` file does not contain information about our subarray data, we need to add a new row to the table with our definitions. Let's first view the first few rows of `OSCNTAB` to see what our new entry needs to look like.

In [20]:
print(oscntable)

CCDAMP CCDCHIP BINX BINY  NX   NY  ... BIASSECTB2  DESCRIPTION  VX1 VX2 VY1 VY2
------ ------- ---- ---- ---- ---- ... ---------- ------------- --- --- --- ---
     C       2    1    1 4144 2068 ...       4126 IDT selection   0   0   0   0
     C       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2070 2046 ...          0 IDT selection   0   0   0   0
     C       2    1    1  534  512 ...          0 IDT selection   0   0   0   0
     D       2    1    1 4144 2068 ...         24 IDT selection   0   0   0   0
     D       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
   ...     ...  ...  ...  ...  ... ...  

We can also choose to just view all of the columns names

In [21]:
print(oscntable.colnames)

['CCDAMP', 'CCDCHIP', 'BINX', 'BINY', 'NX', 'NY', 'TRIMX1', 'TRIMX2', 'TRIMY1', 'TRIMY2', 'BIASSECTA1', 'BIASSECTA2', 'BIASSECTB1', 'BIASSECTB2', 'DESCRIPTION', 'VX1', 'VX2', 'VY1', 'VY2']


Several column names are obvious, but here we define the less obvious ones. 

| Column Name   | Description |
| --- | --- |
| BINX, BINY    | Binning in X and Y. ACS data are never binned, so these will always be 1. |
| TRIMXn        | Number of prescan columns on the left (1) and right (2) sides of the image to remove. |
| TRIMYn        | Number of virtual rows on the bottom (1) and top (2) sides of the image to remove. For subarray data, these are always 0. |
| BIASSECTAn    | Start and end columns to use for the bias level estimation on the left (A) side of the image. |
| BIASSECTBn    | Start and end columns to use for the bias level estimation on the right (B) side of the image. |
| VXn, VYn      | The coordinates of the bottom-left (VX1, VY1) and top-right (VX2, VY2) corners of the virtual overscan region |

The following line sets `chip` to 1 if the subarray is on WFC1, and 2 if the subarray is on WFC2.

In [22]:
chip = 1 if amp in ['A', 'B'] else 2

For the BIASSECTAn and BIASSECTBn values, we want to use the six columns of prescan nearest the exposed area of the CCD. 

In [23]:
bias_a = [0, 0]
if leading > 0:
    bias_a[1] = leading
    bias_a[0] = leading-5 if leading > 5 else 0
        
bias_b = [0, 0]
if trailing > 0:
    bias_b[0] = xsize+1
    bias_b[1] = xsize+5 if trailing > 5 else xsize+trailing

Now we can define a new row for our settings. For subarray data, as there is no virtual overscan, the VXn and VYn values will always be 0. Here we use a dictionary to explicitly define the new values.

In [24]:
new_row = {'CCDAMP': amp,
           'CCDCHIP': chip, 
           'BINX': 1, 
           'BINY': 1, 
           'NX': xsize,
           'NY': ysize, 
           'TRIMX1': leading, 
           'TRIMX2': trailing, 
           'TRIMY1': 0, 
           'TRIMY2': 0, 
           'BIASSECTA1': bias_a[0], 
           'BIASSECTA2': bias_a[1], 
           'BIASSECTB1': bias_b[0], 
           'BIASSECTB2': bias_b[1], 
           'DESCRIPTION': 'Custom OSCN', 
           'VX1': 0, 
           'VX2': 0, 
           'VY1': 0, 
           'VY2': 0}

Now we need to open the custom `OSCTNAB` file and update the table to have our new definition. We will also need to update the raw FITS file to point to the new custom `OSCNTAB` reference file.

In [25]:
oscntable.add_row(new_row)
print(oscntable)

CCDAMP CCDCHIP BINX BINY  NX   NY  ... BIASSECTB2  DESCRIPTION  VX1 VX2 VY1 VY2
------ ------- ---- ---- ---- ---- ... ---------- ------------- --- --- --- ---
     C       2    1    1 4144 2068 ...       4126 IDT selection   0   0   0   0
     C       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2070 2046 ...          0 IDT selection   0   0   0   0
     C       2    1    1  534  512 ...          0 IDT selection   0   0   0   0
     D       2    1    1 4144 2068 ...         24 IDT selection   0   0   0   0
     D       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
   ...     ...  ...  ...  ...  ... ...  

Now we have an idential FITS rec table, but with an additional row for our new information.

In [26]:
oscnrec_new = fits.FITS_rec.from_columns(oscnrec.columns, 
                                         nrows=len(oscntable))
print(Table(oscnrec_new))

CCDAMP CCDCHIP BINX BINY  NX   NY  ... BIASSECTB2  DESCRIPTION  VX1 VX2 VY1 VY2
------ ------- ---- ---- ---- ---- ... ---------- ------------- --- --- --- ---
     C       2    1    1 4144 2068 ...       4126 IDT selection   0   0   0   0
     C       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2070 2046 ...          0 IDT selection   0   0   0   0
     C       2    1    1  534  512 ...          0 IDT selection   0   0   0   0
     D       2    1    1 4144 2068 ...         24 IDT selection   0   0   0   0
     D       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
   ...     ...  ...  ...  ...  ... ...  

Let's populate that last row with our new data!

In [27]:
oscnrec_new[-1] = tuple(oscntable[-1])

Reprint the table and check that we have entered it correctly.

In [28]:
print(Table(oscnrec_new))

CCDAMP CCDCHIP BINX BINY  NX   NY  ... BIASSECTB2  DESCRIPTION  VX1 VX2 VY1 VY2
------ ------- ---- ---- ---- ---- ... ---------- ------------- --- --- --- ---
     C       2    1    1 4144 2068 ...       4126 IDT selection   0   0   0   0
     C       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
     C       2    1    1 2070 2046 ...          0 IDT selection   0   0   0   0
     C       2    1    1  534  512 ...          0 IDT selection   0   0   0   0
     D       2    1    1 4144 2068 ...         24 IDT selection   0   0   0   0
     D       2    1    1 2072 2068 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072 1024 ...          0 IDT selection   0   0   0   0
     D       2    1    1 2072  512 ...          0 IDT selection   0   0   0   0
   ...     ...  ...  ...  ...  ... ...  

Now we can open up our oscn table and replace the old fits rec with our new one

In [29]:
with fits.open(path_to_oscn, mode='update') as hdu:
    hdu[1].data = oscnrec_new

Using the same check we did earlier, we can use `check_oscntab` to see if our settings are defined. If everything was done correctly, the following line should print "True"!

In [30]:
supported = utils_calib.check_oscntab(path_to_oscn, amp, xsize, ysize, leading, trailing)

print('Defined in OSCNTAB? {}'.format(supported))

Defined in OSCNTAB? True


<a id="about"></a>
## About this Notebook
**Curator:** David Stark, ACS Instrument Team, Space Telescope Science Institute

### For more help:

More details may be found on the [ACS website](http://www.stsci.edu/hst/instrumentation/acs) and in the [ACS Instrument](https://hst-docs.stsci.edu/display/ACSIHB) and [Data Handbooks](https://hst-docs.stsci.edu/acsdhb).

Please visit the [HST Help Desk](http://hsthelp.stsci.edu). Through the help desk portal, you can explore the HST Knowledge Base and request additional help from experts.

<hr>

[Top of Page](#titleImageReduction)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 
<br></br>
<br></br>